In [ ]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread as gs
from gspread_dataframe import set_with_dataframe

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/telkomprojectpost-442206-7d1a9ddc7907.json', scope)
gc = gs.authorize(credentials)

In [ ]:
# Membaca data dari Google Sheets
spreadsheet = gc.open('Sementara')
worksheet = spreadsheet.worksheet('PT TLKM')
existing_data = worksheet.get_all_values()

# Konversi ke DataFrame
df = pd.DataFrame(existing_data[1:], columns=existing_data[0])

In [ ]:
# Inisialisasi Sentiment Intensity Analyzer
analyzer = SentimentIntensityAnalyzer()

# Fungsi untuk mendapatkan skor sentimen
def get_sentiment_scores(content):
    return analyzer.polarity_scores(content)

# Mengambil kolom 'content' dan menghitung skor sentimen
df['sentiment_scores'] = df['content'].apply(get_sentiment_scores)

# Menambahkan kolom untuk negative, neutral, positive, dan compound score
df['negative'] = df['sentiment_scores'].apply(lambda x: x['neg'])
df['neutral'] = df['sentiment_scores'].apply(lambda x: x['neu'])
df['positive'] = df['sentiment_scores'].apply(lambda x: x['pos'])
df['compound_score'] = df['sentiment_scores'].apply(lambda x: x['compound'])

# Menghapus kolom 'sentiment_scores' karena sudah tidak diperlukan
df.drop(columns=['sentiment_scores'], inplace=True)

# Update result
ws = gc.open('Sementara').worksheet('PT TLKM')
ws.clear()
set_with_dataframe(ws, df)